In [1]:
# !pip install pomegranate
!pip install pgmpy

     |████████████████████████████████| 337kB 11.1MB/s 


In [2]:
from google.colab import drive
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.linear_model import LogisticRegression
# from pomegranate import BayesianNetwork
from pgmpy.models import BayesianModel
from sklearn.ensemble import ExtraTreesClassifier

In [42]:
drive.mount('/content/gdrive')
root_path = '/content/gdrive/MyDrive/Colab Notebooks/ML_Project/Data/'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [43]:

#loading training data 

events = pd.read_csv(root_path+'events.csv')


In [44]:
shots = events[events.event_type==1]   
#event_type  = 1 = attempt


In [45]:
len(shots)   #Length od input data 

229135

In [46]:
data = pd.get_dummies(shots, columns=['assist_method', 'situation','location', 'bodypart'])     #Taking important features 
data.drop(data.columns[0:17], axis = 1, inplace = True)


In [47]:
a = ['fast_break','no_assist', 'assist_pass', 'assist_cross', 'assist_header',
                'assist_through_ball','open_play', 'set_piece', 'corner', 'free_kick', 'loc_centre_box', 'loc_diff_angle_lr', 'diff_angle_left', 'diff_angle_right',
                'left_side_box', 'left_side_6ybox', 'right_side_box', 'right_side_6ybox', 'close_range',
                'penalty', 'outside_box', 'long_range', 'more_35y', 'more_40y', 'not_recorded', 'right_foot', 
                'left_foot', 'header'  ]
data.columns

Index(['fast_break', 'assist_method_0', 'assist_method_1', 'assist_method_2',
       'assist_method_3', 'assist_method_4', 'situation_1.0', 'situation_2.0',
       'situation_3.0', 'situation_4.0', 'location_3.0', 'location_6.0',
       'location_7.0', 'location_8.0', 'location_9.0', 'location_10.0',
       'location_11.0', 'location_12.0', 'location_13.0', 'location_14.0',
       'location_15.0', 'location_16.0', 'location_17.0', 'location_18.0',
       'location_19.0', 'bodypart_1.0', 'bodypart_2.0', 'bodypart_3.0'],
      dtype='object')

In [48]:
data.columns = ['fast_break','no_assist', 'assist_pass', 'assist_cross', 'assist_header',
                'assist_through_ball','open_play', 'set_piece', 'corner', 'free_kick', 'loc_centre_box', 'loc_diff_angle_lr', 'diff_angle_left', 'diff_angle_right',
                'left_side_box', 'left_side_6ybox', 'right_side_box', 'right_side_6ybox', 'close_range',
                'penalty', 'outside_box', 'long_range', 'more_35y', 'more_40y', 'not_recorded', 'right_foot', 
                'left_foot', 'header'  ]
data['is_goal'] = shots['is_goal']

In [49]:
data.drop('not_recorded', axis=1, inplace=True)

In [50]:
x = data.iloc[:,0:28]
y = data.iloc[:,-1]
x.drop('is_goal', axis=1, inplace=True)

In [51]:

model = ExtraTreesClassifier(n_estimators=10)
model.fit(x, y)
imp = model.feature_importances_ > 0.04               

In [52]:
valid_col = imp.tolist()

In [53]:
init_data = data.copy()
data.head()

,fast_break,no_assist,assist_pass,assist_cross,assist_header,assist_through_ball,open_play,set_piece,corner,free_kick,loc_centre_box,loc_diff_angle_lr,diff_angle_left,diff_angle_right,left_side_box,left_side_6ybox,right_side_box,right_side_6ybox,close_range,penalty,outside_box,long_range,more_35y,more_40y,right_foot,left_foot,header,is_goal
0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
11,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
13,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1
14,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
17,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0


In [54]:
for i, col in enumerate(data.columns):
  if i < len(valid_col) and valid_col[i] == False:
    # print("done")
    init_data.drop(col, axis=1, inplace=True)
  

In [16]:
init_data.head()

,fast_break,loc_centre_box,close_range,penalty,outside_box,is_goal
0,0,0,0,0,0,0
11,0,0,0,0,1,0
13,0,0,0,0,0,1
14,0,0,0,0,1,0
17,0,0,0,0,1,0


In [21]:
x = init_data.iloc[:,0:28]
y = init_data.iloc[:,-1]
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.01, random_state = 1)

In [22]:
#Creating Test and Train data
x_train.drop('is_goal', axis=1, inplace=True)
x_test.drop('is_goal', axis=1, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [23]:
train = x_train.copy()
train['is_goal'] = y_train

### Bayesian Network

In [26]:
import pgmpy

from pgmpy.estimators import ParameterEstimator, BayesianEstimator
from pgmpy.models import BayesianModel
modelRelations = []
#adding edges 
modelRelations.append(('penalty','loc_centre_box'))
modelRelations.append(('close_range','loc_centre_box'))
modelRelations.append(('loc_centre_box','is_goal'))
modelRelations.append(('outside_box','is_goal'))
modelRelations.append(('fast_break','is_goal'))


In [29]:
#Bayesian Model
model1 = BayesianModel(modelRelations) 
model1.fit(train, estimator=BayesianEstimator, prior_type="BDeu", )

In [30]:
#evidence 
def createsEvidence(df_point):

    evidence = {}
    for col in x_test.columns:
      evidence[col] = df_point[col]
    
    return evidence

In [31]:
#Inference Model
from pgmpy.inference import VariableElimination

predictions = {}

for pred_idx in range(len(x_test)):
  print(pred_idx)
  infer = VariableElimination(model1)
  evidence = createsEvidence(x_test.iloc[pred_idx])
  result = infer.query(['is_goal'], evidence = evidence)
  predictions[pred_idx] = (result.values[1], y_test.iloc[pred_idx])

Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]
Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]
Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]
Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]
Finding Elimination Order: : : 0it [00:00, ?it/s]

0
1
2
3
4



0it [00:00, ?it/s]
Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]
Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]
Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]
Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]


5
6
7
8
9


Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]
Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]
Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]
Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]
Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]


...

Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]
Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]
Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]
Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]
Finding Elimination Order: : : 0it [00:00, ?it/s]


2284
2285
2286
2287


0it [00:00, ?it/s]
Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]
Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]
Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]


2288
2289
2290
2291


Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]


In [32]:
#Confusion Matrix
tp = 0  #true positive
tn = 0  #true negative
fp = 0  #false negative
fn = 0  #false negative 

for pred_idx in range(len(y_test)):
  if predictions[pred_idx][0] > 0.5 and predictions[pred_idx][1] == 1:
    tp += 1
  if predictions[pred_idx][0] > 0.5 and predictions[pred_idx][1] == 0:
    fp += 1
  if predictions[pred_idx][0] <= 0.5 and predictions[pred_idx][1] == 0:
    tn += 1
  if predictions[pred_idx][0] <= 0.5 and predictions[pred_idx][1] == 1:
    fn += 1



In [33]:
total = tp + tn + fp + fn
tp, tn, fp, fn

(6, 2026, 9, 251)

In [35]:
#Accuracy for Bayesian Networks
accuracy = (tp + tn) / total
accuracy

0.8865619546247818

## Logistic Regression

In [38]:
lr = LogisticRegression(max_iter = 50, penalty='l2')
lr.fit(x_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=50,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [39]:
sc = lr.score(x_test, y_test)
print(sc)

0.8944153577661431


In [40]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
print('test contains {} shots and {} are goals.'.format(len(y_test), y_test.sum()))
print('Accuracy {}%.'.format(round(lr.score(x_test, y_test)*100),2))
print(' ROC-AUC of {}%'.format(round(roc_auc_score(y_test, lr.predict_proba(x_test)[:, 1])*100),2))
print(confusion_matrix(y_test,lr.predict(x_test)))

test contains 2292 shots and 257 are goals.
Accuracy 89%.
 ROC-AUC of 75%
[[2029    6]
 [ 236   21]]
